In [1]:
import sys
import time
sys.path.insert(0, '/home/anish/ASC_ML')

In [2]:
import pandas as pd
import dask.dataframe as dd
import time

In [3]:
from ASC_ML.preprocessing import dataset_container as dc
from ASC_ML.preprocessing import data_cleaning as data_cl
from ASC_ML.networkbuilding import multiple_model_gen_v3 as multiple
from ASC_ML.networkbuilding import dataframe_extractor as de
from ASC_ML.networkbuilding import model_generation as model_gen
from ASC_ML.networkbuilding import model_optimization as model_opt
from ASC_ML.networkbuilding import model_stacking
from ASC_ML.networkbuilding import final

2022-09-07 23:59:56.035364: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [4]:
df = dd.read_csv("/home/anish/Downloads/autokeras-regression-master/Data.csv")

In [5]:
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [6]:
target_col = "sales"
X = df.loc[:, df.columns != target_col]
y = df.loc[:, target_col]

In [7]:
data_cl_obj = data_cl.DataCleaning(["sales"], df)

In [8]:
data_cl_obj.col_info

{'TV': {'dtype': dtype('float64'),
  'is_label': False,
  'missing': 0.0,
  'cardinality': 190},
 'radio': {'dtype': dtype('float64'),
  'is_label': False,
  'missing': 0.0,
  'cardinality': 167},
 'newspaper': {'dtype': dtype('float64'),
  'is_label': False,
  'missing': 0.0,
  'cardinality': 172},
 'sales': {'dtype': dtype('float64'),
  'is_label': True,
  'missing': 0.0,
  'cardinality': 121}}

In [9]:
data_cl_obj.dataset.get()[0].shape[1]

4

In [10]:
X_columns = []
Y_columns = []
Y_cardinalities = []
for column_name in data_cl_obj.col_info:
    if data_cl_obj.col_info[column_name]["is_label"] == True:
        Y_columns.append(column_name)
        Y_cardinalities.append(data_cl_obj.col_info[column_name]["cardinality"])
    else:
        X_columns.append(column_name)

X = data_cl_obj.dataset.get()[0][X_columns]
Y = data_cl_obj.dataset.get()[0][Y_columns]

In [11]:
Y.head()

,sales
0,22.1
1,10.4
2,9.3
3,18.5
4,12.9


In [12]:
total = dd.concat([X,Y], axis = 1)

/home/anish/anaconda3/envs/tfgpu/lib/python3.9/site-packages/dask/dataframe/multi.py:1254: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indices of each dataframes are 
 aligned. This assumption is not generally safe.
  warnings.warn(


In [13]:
total.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [14]:
data_cl_obj.dataset.get()[0].compute()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9
...,...,...,...,...
195,38.2,3.7,13.8,7.6
196,94.2,4.9,8.1,9.7
197,177.0,9.3,6.4,12.8
198,283.6,42.0,66.2,25.5


In [15]:
data_cl_obj.dataset.get()[0]["sales"].compute()

0      22.1
1      10.4
2       9.3
3      18.5
4      12.9
       ... 
195     7.6
196     9.7
197    12.8
198    25.5
199    13.4
Name: sales, Length: 200, dtype: float64

In [16]:
directory_train = "/home/anish/ASC-ML-EXP-DATASETS/LinReg-tabular/house-prices-advanced-regression-techniques/train.csv"
directory_test = "/home/anish/ASC-ML-EXP-DATASETS/LinReg-tabular/house-prices-advanced-regression-techniques/test.csv"
train_df = dd.read_csv(directory_train, assume_missing = True, sample_rows=1000)
test_df = dd.read_csv(directory_test, assume_missing = True, sample_rows=1000)

In [17]:
train_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1.0,60.0,RL,65.0,8450.0,Pave,NaN,Reg,Lvl,AllPub,...,0.0,NaN,NaN,NaN,0.0,2.0,2008.0,WD,Normal,208500.0
1,2.0,20.0,RL,80.0,9600.0,Pave,NaN,Reg,Lvl,AllPub,...,0.0,NaN,NaN,NaN,0.0,5.0,2007.0,WD,Normal,181500.0
2,3.0,60.0,RL,68.0,11250.0,Pave,NaN,IR1,Lvl,AllPub,...,0.0,NaN,NaN,NaN,0.0,9.0,2008.0,WD,Normal,223500.0
3,4.0,70.0,RL,60.0,9550.0,Pave,NaN,IR1,Lvl,AllPub,...,0.0,NaN,NaN,NaN,0.0,2.0,2006.0,WD,Abnorml,140000.0
4,5.0,60.0,RL,84.0,14260.0,Pave,NaN,IR1,Lvl,AllPub,...,0.0,NaN,NaN,NaN,0.0,12.0,2008.0,WD,Normal,250000.0


In [18]:
data_cl_obj = data_cl.DataCleaning(["SalePrice"], train_dataframe = train_df, test_dataframe = test_df)

In [19]:
data_cl_obj.col_info

{'Id': {'dtype': dtype('float64'),
  'is_label': False,
  'missing': 0.0,
  'cardinality': 1460},
 'MSSubClass': {'dtype': dtype('float64'),
  'is_label': False,
  'missing': 0.0,
  'cardinality': 15},
 'MSZoning': {'dtype': dtype('O'),
  'is_label': False,
  'missing': 0.0,
  'cardinality': 5},
 'LotFrontage': {'dtype': dtype('float64'),
  'is_label': False,
  'missing': 0.1773972602739726,
  'cardinality': 110},
 'LotArea': {'dtype': dtype('float64'),
  'is_label': False,
  'missing': 0.0,
  'cardinality': 1073},
 'Street': {'dtype': dtype('O'),
  'is_label': False,
  'missing': 0.0,
  'cardinality': 2},
 'Alley': {'dtype': dtype('O'),
  'is_label': False,
  'missing': 0.9376712328767123,
  'cardinality': 2},
 'LotShape': {'dtype': dtype('O'),
  'is_label': False,
  'missing': 0.0,
  'cardinality': 4},
 'LandContour': {'dtype': dtype('O'),
  'is_label': False,
  'missing': 0.0,
  'cardinality': 4},
 'Utilities': {'dtype': dtype('O'),
  'is_label': False,
  'missing': 0.0,
  'cardinal

In [20]:
data_cl_obj.encode()

In [21]:
data_cl_obj.dataset.get()[0].compute()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1.0,60.0,0,65.0,8450.0,0,0,0,0,0,...,0.0,0,0,0,0.0,2.0,2008.0,0,0,208500.0
1,2.0,20.0,0,80.0,9600.0,0,0,0,0,0,...,0.0,0,0,0,0.0,5.0,2007.0,0,0,181500.0
2,3.0,60.0,0,68.0,11250.0,0,0,1,0,0,...,0.0,0,0,0,0.0,9.0,2008.0,0,0,223500.0
3,4.0,70.0,0,60.0,9550.0,0,0,1,0,0,...,0.0,0,0,0,0.0,2.0,2006.0,0,1,140000.0
4,5.0,60.0,0,84.0,14260.0,0,0,1,0,0,...,0.0,0,0,0,0.0,12.0,2008.0,0,0,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456.0,60.0,0,62.0,7917.0,0,0,0,0,0,...,0.0,0,0,0,0.0,8.0,2007.0,0,0,175000.0
1456,1457.0,20.0,0,85.0,13175.0,0,0,0,0,0,...,0.0,0,1,0,0.0,2.0,2010.0,0,0,210000.0
1457,1458.0,70.0,0,66.0,9042.0,0,0,0,0,0,...,0.0,0,3,1,2500.0,5.0,2010.0,0,0,266500.0
1458,1459.0,20.0,0,68.0,9717.0,0,0,0,0,0,...,0.0,0,0,0,0.0,4.0,2010.0,0,0,142125.0


In [22]:
data_cl_obj.dataset.get(types = ["train"])[0].compute()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1.0,60.0,0,65.0,8450.0,0,0,0,0,0,...,0.0,0,0,0,0.0,2.0,2008.0,0,0,208500.0
1,2.0,20.0,0,80.0,9600.0,0,0,0,0,0,...,0.0,0,0,0,0.0,5.0,2007.0,0,0,181500.0
2,3.0,60.0,0,68.0,11250.0,0,0,1,0,0,...,0.0,0,0,0,0.0,9.0,2008.0,0,0,223500.0
3,4.0,70.0,0,60.0,9550.0,0,0,1,0,0,...,0.0,0,0,0,0.0,2.0,2006.0,0,1,140000.0
4,5.0,60.0,0,84.0,14260.0,0,0,1,0,0,...,0.0,0,0,0,0.0,12.0,2008.0,0,0,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456.0,60.0,0,62.0,7917.0,0,0,0,0,0,...,0.0,0,0,0,0.0,8.0,2007.0,0,0,175000.0
1456,1457.0,20.0,0,85.0,13175.0,0,0,0,0,0,...,0.0,0,1,0,0.0,2.0,2010.0,0,0,210000.0
1457,1458.0,70.0,0,66.0,9042.0,0,0,0,0,0,...,0.0,0,3,1,2500.0,5.0,2010.0,0,0,266500.0
1458,1459.0,20.0,0,68.0,9717.0,0,0,0,0,0,...,0.0,0,0,0,0.0,4.0,2010.0,0,0,142125.0


In [23]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

In [24]:
# X = data_cl_obj.dataset.get(types = ["train"])[0].compute()
X = data_cl_obj.dataset.get(types = ["train"])[0]
X = X.dropna().copy()
Y = X["SalePrice"]

In [25]:
X.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1.0,60.0,0,65.0,8450.0,0,0,0,0,0,...,0.0,0,0,0,0.0,2.0,2008.0,0,0,208500.0
1,2.0,20.0,0,80.0,9600.0,0,0,0,0,0,...,0.0,0,0,0,0.0,5.0,2007.0,0,0,181500.0
2,3.0,60.0,0,68.0,11250.0,0,0,1,0,0,...,0.0,0,0,0,0.0,9.0,2008.0,0,0,223500.0
3,4.0,70.0,0,60.0,9550.0,0,0,1,0,0,...,0.0,0,0,0,0.0,2.0,2006.0,0,1,140000.0
4,5.0,60.0,0,84.0,14260.0,0,0,1,0,0,...,0.0,0,0,0,0.0,12.0,2008.0,0,0,250000.0


In [26]:
type(X)

dask.dataframe.core.DataFrame

In [27]:
type(Y)

dask.dataframe.core.Series

In [28]:
# X = data_cl_obj.dataset.get(types = ["train"])[0].compute()
# Y = data_cl_obj.dataset.get(types = ["train"])[0]["SalePrice"].compute()
start = time.time()
model = RandomForestClassifier()
n = 50
rfe = RFE(estimator = model, n_features_to_select=n)
fit = rfe.fit(X, Y)
X = X[X.columns[fit.support_]]
X.head()
end = time.time()

In [29]:
X.head()

,Id,MSSubClass,LotFrontage,LotArea,LotShape,LotConfig,Neighborhood,Condition1,HouseStyle,OverallQual,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,ScreenPorch,Fence,MoSold,YrSold,SaleType,SalePrice
0,1.0,60.0,65.0,8450.0,0,0,0,0,0,7.0,...,548.0,0.0,61.0,0.0,0.0,0,2.0,2008.0,0,208500.0
1,2.0,20.0,80.0,9600.0,0,1,1,1,1,6.0,...,460.0,298.0,0.0,0.0,0.0,0,5.0,2007.0,0,181500.0
2,3.0,60.0,68.0,11250.0,1,0,0,0,0,7.0,...,608.0,0.0,42.0,0.0,0.0,0,9.0,2008.0,0,223500.0
3,4.0,70.0,60.0,9550.0,1,2,2,0,0,7.0,...,642.0,0.0,35.0,272.0,0.0,0,2.0,2006.0,0,140000.0
4,5.0,60.0,84.0,14260.0,1,1,3,0,0,8.0,...,836.0,192.0,84.0,0.0,0.0,0,12.0,2008.0,0,250000.0


In [30]:
import numpy as np
np_Y = np.asarray(Y)
np_X = np.asarray(X)

In [31]:
np_Y = np_Y.reshape(-1,1)
np_Y.shape

(1121, 1)

In [32]:
print("Time Taken = ", (end-start))

Time Taken =  79.81039500236511


In [33]:
# from sklearn.preprocessing import MinMaxScaler
# scaler_x = MinMaxScaler()
# scaler_x.fit(X)
# train_X_scaled = scaler_x.transform(X)

In [34]:
from sklearn.preprocessing import MinMaxScaler
scaler_x = MinMaxScaler()
scaler_x.fit(np_X)
train_X_scaled = scaler_x.transform(np_X)

scaler_y = MinMaxScaler()
scaler_y.fit(np_Y)
train_Y_scaled = scaler_y.transform(np_Y)

In [35]:
new_X = dd.from_array(train_X_scaled)

In [36]:
new_X.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.000000,0.235294,0.150685,0.033420,0.000000,0.00,0.000000,0.000,0.000000,0.625,...,0.308426,0.000000,0.111517,0.000000,0.0,0.0,0.090909,0.50,0.0,0.240644
1,0.000685,0.000000,0.202055,0.038795,0.000000,0.25,0.041667,0.125,0.142857,0.500,...,0.238474,0.347725,0.000000,0.000000,0.0,0.0,0.363636,0.25,0.0,0.203128
2,0.001371,0.235294,0.160959,0.046507,0.333333,0.00,0.000000,0.000,0.000000,0.625,...,0.356121,0.000000,0.076782,0.000000,0.0,0.0,0.727273,0.50,0.0,0.261487
3,0.002056,0.294118,0.133562,0.038561,0.333333,0.50,0.083333,0.000,0.000000,0.625,...,0.383148,0.000000,0.063985,0.492754,0.0,0.0,0.090909,0.00,0.0,0.145464
4,0.002742,0.235294,0.215753,0.060576,0.333333,0.25,0.125000,0.000,0.000000,0.750,...,0.537361,0.224037,0.153565,0.000000,0.0,0.0,1.000000,0.50,0.0,0.298308


In [37]:
total_data = np.append(train_X_scaled, train_Y_scaled, axis = 1)
from sklearn.model_selection import train_test_split
(train, test) = train_test_split(total_data, test_size=0.25, random_state=42)
X_train = train[:,:58]
y_train = train[:,-1]
X_test = test[:,:58]
y_test = test[:,-1]

In [38]:
y_train = y_train.reshape((y_train.shape[0],1))
y_test = y_test.reshape((y_test.shape[0],1))

In [39]:
X_train.shape

(840, 51)

In [ ]:
start = time.time()
# m = multiple.Multiple_Model_Gen_V3(X_train, y_train, X_test, y_test, 50, 64, input_shape = 51, 
#                                    max_no_layers = 3, model_per_batch = 10, 
#                                    save_dir = "/home/anish/ASC_ML_test_weights/")
# m.get_best_models(save = False)
# opt = model_opt.Model_Optimization(X_train, y_train, X_test, y_test, 200, m.evaluate_dict_list, save_dir = "/home/anish/ASC_ML_test_weights/candidate_models/")
# opt.optimize_models(save=True)
# stacked_models = model_stacking.Model_Stacking(X_train, y_train, X_test, y_test, opt.saved_paths, opt.model_confs, save_dir = "/home/anish/ASC_ML_test_weights/stacked_models/")
# stacked_models.optimize_stacked_models()
f = final.Final(X_train, y_train, X_test, y_test, 50, 64, input_shape = 51, 
                                   max_no_layers = 3, model_per_batch = 10, 
                                   save_dir = "/home/anish/ASC_ML_test_weights/")
f.get_all_best_models()

end = time.time()
print("Time Taken : ", (end-start))

2022-09-08 00:01:25.845538: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-09-08 00:01:25.847286: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-09-08 00:01:25.916805: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-08 00:01:25.917134: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2060 computeCapability: 7.5
coreClock: 1.695GHz coreCount: 30 deviceMemorySize: 5.79GiB deviceMemoryBandwidth: 312.97GiB/s
2022-09-08 00:01:25.917197: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-09-08 00:01:25.919728: I tensorflow/stream_executor/platform/de


 
 

loss  :  0.0796085074543953 , TEST :  0.1747775375843048
output_layer_dense_32_32_loss  :  0.009345349855720997 , TEST :  0.016955643892288208
output_layer_dense_32_64_loss  :  0.0104688024148345 , TEST :  0.018715886399149895
output_layer_dense_32_128_loss  :  0.007614535745233297 , TEST :  0.018087998032569885
output_layer_dense_32_256_loss  :  0.00878751091659069 , TEST :  0.018003208562731743
output_layer_dense_32_512_loss  :  0.007440795190632343 , TEST :  0.018543614074587822
output_layer_dense_32_1024_loss  :  0.008211552165448666 , TEST :  0.016072211787104607
output_layer_dense_64_32_loss  :  0.007366647478193045 , TEST :  0.01738058589398861
output_layer_dense_64_64_loss  :  0.007999608293175697 , TEST :  0.020363906398415565
output_layer_dense_64_128_loss  :  0.006461154669523239 , TEST :  0.015453074127435684
output_layer_dense_64_256_loss  :  0.0059125544503331184 , TEST :  0.015201414003968239

 
 

loss  :  0.08356492966413498 , TEST :  0.15585288405418396
output_l


 
 

loss  :  0.08370291441679001 , TEST :  0.18353743851184845
output_layer_dense_64_256_64_loss  :  0.012077492661774158 , TEST :  0.01945473626255989
output_layer_dense_64_256_128_loss  :  0.007012433838099241 , TEST :  0.01850683055818081
output_layer_dense_64_256_256_loss  :  0.012891602702438831 , TEST :  0.021974196657538414
output_layer_dense_64_256_512_loss  :  0.010754144750535488 , TEST :  0.019411642104387283
output_layer_dense_64_256_1024_loss  :  0.0058311899192631245 , TEST :  0.014276190660893917
output_layer_dense_64_512_32_loss  :  0.00521858548745513 , TEST :  0.0157313235104084
output_layer_dense_64_512_64_loss  :  0.007066431920975447 , TEST :  0.018878567963838577
output_layer_dense_64_512_128_loss  :  0.005876491777598858 , TEST :  0.017239386215806007
output_layer_dense_64_512_256_loss  :  0.005875939503312111 , TEST :  0.019561316817998886
output_layer_dense_64_512_512_loss  :  0.011098602786660194 , TEST :  0.01850324682891369

 
 

loss  :  0.061352454125881


 
 

loss  :  0.06753197312355042 , TEST :  0.09517157822847366
output_layer_dense_1024_32_32_loss  :  0.00531650660559535 , TEST :  0.009344001300632954
output_layer_dense_1024_64_32_loss  :  0.003132834332063794 , TEST :  0.006260573863983154
output_layer_dense_1024_64_64_loss  :  0.003763905493542552 , TEST :  0.006910009775310755
output_layer_dense_1024_128_32_loss  :  0.011715373955667019 , TEST :  0.013765119947493076
output_layer_dense_1024_128_64_loss  :  0.008857171051204205 , TEST :  0.0114750349894166
output_layer_dense_1024_128_128_loss  :  0.007946559228003025 , TEST :  0.010291540995240211
output_layer_dense_1024_256_32_loss  :  0.005541261751204729 , TEST :  0.008472610265016556
output_layer_dense_1024_256_64_loss  :  0.0051927147433161736 , TEST :  0.009174226783216
output_layer_dense_1024_256_128_loss  :  0.007867391221225262 , TEST :  0.009633073583245277
output_layer_dense_1024_256_256_loss  :  0.00819826778024435 , TEST :  0.009845389984548092

 
 

loss  :  0.0632

2022-09-08 00:04:29.183609: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /home/anish/ASC_ML_test_weights/candidate_models/dense_1024_64_32/assets
--------------------------------------------------------------------------------
Model Name : dense_1024_64_64

BEST HYPERPARAMETERS : BEST_LOSS : 0.011227564886212349, BEST_ACTIVATION : selu, BEST_INITIALIZER : LecunNormal, BEST_LEARINING_RATE : 0.00013, BEST_BATCHSIZE : 64

 
 

loss  :  0.020252225920557976 , TEST :  0.02266616001725197
INFO:tensorflow:Assets written to: /home/anish/ASC_ML_test_weights/candidate_models/dense_1024_64_64/assets
--------------------------------------------------------------------------------
Model Name : dense_1024_32

